# AI 품질 평가 모델 학습 - KLUE-RoBERTa-large
이 노트북은 AI 응답 품질 평가를 위한 Multi-label Classification 모델을 학습합니다.

평가 기준 (9개):

*   linguistic_acceptability (언어적 수용성)
*   consistency (일관성)
*   interestingness (흥미로움)
*   unbias (편향 없음)
*   harmlessness (무해성)
*   no_hallucination (환각 없음)
*   understandability (이해 가능성)
*   sensibleness (합리성)
*   specificity (구체성)


# 1. 라이브러리 설치 및 임포트

In [2]:
!pip install torch transformers datasets pandas scikit-learn tqdm -q

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [60]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.auto import tqdm
from torch.utils.data import Dataset, Sampler
import warnings

warnings.filterwarnings('ignore')

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 설정

In [61]:
# 모델 및 학습 설정
MODEL_NAME = "klue/roberta-base"
MAX_LENGTH = 256
BATCH_SIZE = 128
LEARNING_RATE = 1e-5
NUM_EPOCHS = 3
SEED = 42

# 평가 기준 (타겟 컬럼)
CRITERIA = [
    'linguistic_acceptability',
    'consistency',
    'interestingness',
    'unbias',
    'harmlessness',
    'no_hallucination',
    'understandability',
    'sensibleness',
    'specificity'
]
NUM_LABELS = len(CRITERIA)

# 시드 고정
torch.manual_seed(SEED)
np.random.seed(SEED)

In [62]:
train_df = pd.read_csv('/content/drive/MyDrive/ai_허브_데이터셋/data/train/training_all_aggregated.csv', encoding='utf-8-sig')
val_df = pd.read_csv('/content/drive/MyDrive/ai_허브_데이터셋/data/val/validation_all_aggregated.csv', encoding='utf-8-sig')

In [63]:
print(f"Training 데이터: {len(train_df):,}개")
print(f"Validation 데이터: {len(val_df):,}개")

Training 데이터: 400,572개
Validation 데이터: 50,047개


In [64]:
print(train_df.iloc[0])

source_file                                                         경제활동_상품상거래_209.json
conversation_id                                                                     864
topic                                                                      경제활동, 상품/상거래
num_evaluators                                                                        5
exchange_id                                                                     c864.e1
utterance_id                                                                    c864.u2
utterance_index                                                                       1
num_evaluations                                                                       3
human_question                                         저번 주 포스코홀딩스의 실적이 나왔는데, 어떤 내용이었어?
bot_response                          포스코홀딩스의 3분기 실적은 철강시황 악화와 타이푼 힌남노로 인한 침수 피해로 인해...
bot_response_length                                                                  58
linguistic_acceptability_yes_cou

# 데이터의 투표 수 비율 변환 및 각 샘플의 난이도 수치화

In [65]:
def preprocess_for_curriculum(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1. 텍스트 결측치 처리 및 입력 텍스트 생성
    df['human_question'] = df['human_question'].fillna('')
    df['bot_response'] = df['bot_response'].fillna('')


    df['input_text'] = df['human_question'] + ' [SEP] ' + df['bot_response']

    # 2. Soft Labels 및 개별 난이도 계산
    # 각 행의 '전체 지표 난이도'를 합산할 변수를 초기화
    df['total_difficulty'] = 0

    for c in CRITERIA:
        # 투표 합계 계산
        total_votes = df[f'{c}_yes_count'] + df[f'{c}_no_count'] + 1e-8 #total_votes가 0일때 0으로 나누는 오류를 방지하기 위한 작은값

        # [Soft Label] 찬성 비율 계산 (0.0 ~ 1.0)
        df[f'{c}_soft'] = df[f'{c}_yes_count'] / total_votes

        # [난이도 측정] 만장일치 여부 확인
        # 3:0 혹은 0:3이면 쉬움(0), 2:1 혹은 1:2이면 어려움(1)으로 판단
        # 데이터셋에 이미 'unanimous' 컬럼이 있다면 활용하고, 없다면 직접 계산.
        is_hard = (df[f'{c}_yes_count'] > 0) & (df[f'{c}_no_count'] > 0)
        df['total_difficulty'] += is_hard.astype(int)

    # 3. 최종 난이도 점수 (0점: 매우 쉬움 ~ 9점: 매우 어려움)
    # 이 점수는 나중에 CurriculumSampler에서 학습 순서를 정하는 기준이 됨
    return df

# 전처리 실행
train_df = preprocess_for_curriculum(train_df)
val_df = preprocess_for_curriculum(val_df)

# 생성 결과 확인
print("--- 전처리 결과 샘플 (첫 5행) ---")
display(train_df[['input_text', 'total_difficulty'] + [f'{c}_soft' for c in CRITERIA[:2]]].head())

print(f"\n난이도 분포:\n{train_df['total_difficulty'].value_counts().sort_index()}")

--- 전처리 결과 샘플 (첫 5행) ---


,input_text,total_difficulty,linguistic_acceptability_soft,consistency_soft
0,"저번 주 포스코홀딩스의 실적이 나왔는데, 어떤 내용이었어? [SEP] 포스코홀딩스의...",1,0.0,1.0
1,영업이익도 많이 감소했으려나? [SEP] 영업이익은 71% 감소하여 9천억원이었습ㄴㅣ다.,0,0.0,1.0
2,왜 영업이익이 감소 원인은 뭐라고 지목되고 있는데? [SEP] 영업이익 감소의 원인...,0,1.0,1.0
3,"포스코홀딩스는 이전 분기에 좋은 실적을 올렸었는데, 왜 실적이 감소했을까? [SEP...",0,0.0,1.0
4,"포항제철소가 침수돼 생산이 중단되었다고 들었는데, 이로 인한 영향이 있었겠네. [S...",0,0.0,1.0



난이도 분포:
total_difficulty
0    361997
1     27831
2      8506
3      1714
4       450
5        69
6         5
Name: count, dtype: int64


In [66]:
# 가장 쉬운 데이터 1개 출력
print("--- [Easy] 난이도 0점 샘플 ---")
display(train_df[train_df['total_difficulty'] == 0][['input_text']].head(1))

# 가장 어려운 데이터 1개 출력
max_diff = train_df['total_difficulty'].max()
print(f"\n--- [Hard] 난이도 {max_diff}점 샘플 ---")
display(train_df[train_df['total_difficulty'] == max_diff][['input_text']].head(1))

--- [Easy] 난이도 0점 샘플 ---


,input_text
1,영업이익도 많이 감소했으려나? [SEP] 영업이익은 71% 감소하여 9천억원이었습ㄴㅣ다.



--- [Hard] 난이도 6점 샘플 ---


,input_text
145074,그게 아니라 나는 특이한 이름을 가진 음식이 궁금했어. [SEP] '눈물의 떡'이라...


# 텐서 가공

In [67]:
class MultiHeadSoftDataset(Dataset):
    """Multi-Head와 Soft Label을 지원하는 데이터셋"""
    def __init__(self, df, tokenizer, max_length=256):
        self.df = df.reset_index(drop=True) #0부터 순서대로
        self.tokenizer = tokenizer
        self.max_length = max_length
        # 위에서 만든 _soft컬럼들을 리스트로 저장
        self.target_cols = [f'{c}_soft' for c in CRITERIA]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['input_text']
        # Soft Label들을 텐서로 변환 (0.0 ~ 1.0 사이의 값)
        labels = torch.tensor(row[self.target_cols].values.astype(np.float32), dtype=torch.float)

        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': labels,
            'difficulty': torch.tensor(row['total_difficulty'], dtype=torch.float)
        }

In [68]:
class CurriculumSampler(Sampler):
    """에폭별로 학습 데이터의 난이도를 조절하는 샘플러"""
    def __init__(self, dataset_df, total_epochs):
        self.df = dataset_df
        self.total_epochs = total_epochs
        self.current_epoch = 0

        # 난이도 점수(0~9점)를 미리 가져옵니다.
        self.difficulties = self.df['total_difficulty'].values

    def set_epoch(self, epoch):
        self.current_epoch = epoch

    def __iter__(self):
        # [커리큘럼 전략]
        # 에폭이 지남에 따라 포함할 난이도 임계값(Threshold)을 높입니다.
        # 예: 3에폭 기준
        # Epoch 0: 난이도 0~3점 (쉬운 것만)
        # Epoch 1: 난이도 0~6점 (중간까지)
        # Epoch 2: 난이도 0~9점 (전체 학습)

        max_difficulty_allowed = ((self.current_epoch + 1) / self.total_epochs) * 9

        # 조건에 맞는 데이터의 인덱스만 추출합니다.
        indices = np.where(self.difficulties <= max_difficulty_allowed)[0]

        # 학습을 위해 인덱스를 무작위로 섞습니다.
        np.random.shuffle(indices)

        print(f"\n[Curriculum] Epoch {self.current_epoch}: 난이도 {max_difficulty_allowed:.1f} 이하 데이터 {len(indices)}개 학습")
        return iter(indices.tolist())

    def __len__(self):
        # 현재 난이도 임계값에 맞는 데이터 개수를 반환합니다.
        max_difficulty_allowed = ((self.current_epoch + 1) / self.total_epochs) * 9
        return len(np.where(self.difficulties <= max_difficulty_allowed)[0])

# 디코딩 테스트

In [69]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
test_dataset = MultiHeadSoftDataset(train_df, tokenizer, MAX_LENGTH)

sample = test_dataset[0]

print("--- 1. Decoding Test ---")
decoded_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)
print(f"Decoded (with special tokens):\n{decoded_text[:150]}...")

--- 1. Decoding Test ---
Decoded (with special tokens):
[CLS] 저번 주 포스코홀딩스의 실적이 나왔는데, 어떤 내용이었어? [SEP] 포스코홀딩스의 3분기 실적은 철강시황 악화와 타이푼 힌남노로 인한 침수 피해로 인해 71 % 급감했습니다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA...


# 레이블과 난이도가 잘 들어갔는지 테스트

In [70]:
print("\n--- 2. Label & Difficulty Test ---")
print(f"Label Tensor Shape: {sample['labels'].shape}")
print(f"Label Values: {sample['labels'].tolist()}")
print(f"Difficulty Score: {sample['difficulty'].item()}")

# 원본 데이터와 대조
print(f"Original Row Difficulty: {train_df.iloc[0]['total_difficulty']}")


--- 2. Label & Difficulty Test ---
Label Tensor Shape: torch.Size([9])
Label Values: [0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333432674408]
Difficulty Score: 1.0
Original Row Difficulty: 1


# *`배치사이즈 잘 나왔는지 테스트`*

In [71]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
batch = next(iter(test_loader))

print("\n--- 3. Batch Shape Test ---")
print(f"Batch Input IDs Shape: {batch['input_ids'].shape}")
print(f"Batch Attention Mask Shape: {batch['attention_mask'].shape}")
print(f"Batch Labels Shape: {batch['labels'].shape}")
print(f"Batch Difficulty Shape: {batch['difficulty'].shape}")


--- 3. Batch Shape Test ---
Batch Input IDs Shape: torch.Size([128, 256])
Batch Attention Mask Shape: torch.Size([128, 256])
Batch Labels Shape: torch.Size([128, 9])
Batch Difficulty Shape: torch.Size([128])


# 모델 아키텍처

In [72]:
import torch.nn as nn
from transformers import AutoModel

class MultiHeadModel(nn.Module):
    def __init__(self, model_name='klue/roberta-base', num_criteria=9):
        super().__init__()
        # 1. Shared Backbone: 문맥 이해
        self.encoder = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.encoder.config.hidden_size

        # 2. EDA 기반 그룹 정의
        # Group A: Content, Group B: Safety, Group C: Coherence, Group D: Independent
        self.criteria = CRITERIA # 9개 지표 리스트

        # 3. Task-Specific Heads: 각 지표별(9개ㅋ) 독립적인 판단
        self.heads = nn.ModuleDict({
            criterion: nn.Sequential(
                nn.Linear(self.hidden_size, 256),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(256, 1)
            ) for criterion in self.criteria
        })

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :] # [CLS] 토큰 활용

        # 각 헤드에서 결과 산출
        logits = [self.heads[c](cls_output) for c in self.criteria]
        return torch.cat(logits, dim=-1) # [batch_size, 9]

# custom Multi-loss

In [73]:
class CombinedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # 기본 BCE (Soft Label 지원용)
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

        #기준별 가중치 설정
        self.weights = torch.tensor([
            1.0, # linguistic_acceptability
            1.2, # consistency
            0.8, # interestingness 불균형
            1.0, # unbias
            1.0, # harmlessness
            2.0, # no_hallucination (중요도 및 난이도 최고)
            0.8, # understandability
            1.0, # sensibleness
            0.8  # specificity 불균형
        ]).to(device)

    def forward(self, logits, targets):
        # targets는 1단계에서 만든 soft_label
        loss = self.bce(logits, targets)

        # EDA 가중치 적용
        weighted_loss = loss * self.weights

        return weighted_loss.mean()

# 통합 학습 루프

In [74]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# 1. 모델, 손실함수, 데이터셋 준비
model = MultiHeadModel(MODEL_NAME).to(device)
criterion = CombinedLoss()
train_dataset = MultiHeadSoftDataset(train_df, tokenizer, MAX_LENGTH)
val_dataset = MultiHeadSoftDataset(val_df, tokenizer, MAX_LENGTH)

# 2. 샘플러 및 데이터로더 설정
train_sampler = CurriculumSampler(train_df, total_epochs=NUM_EPOCHS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# 3. 옵티마이저 및 스케줄러 (학습률 관리)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_df) // BATCH_SIZE * NUM_EPOCHS # 실제는 샘플러 때문에 더 적지만 최대치로 설정
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# 4. 학습 루프 구현
def train_model():
    for epoch in range(NUM_EPOCHS):
        # [Curriculum] 에폭 설정 및 데이터로더 갱신
        train_sampler.set_epoch(epoch)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)

        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward
            logits = model(input_ids, attention_mask)

            # Loss 계산 (EDA 가중치 반영된 CombinedLoss)
            loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch} Average Loss: {avg_loss:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 평기지표 계산

In [75]:
from sklearn.metrics import f1_score

def calculate_metrics(logits, labels):
    preds = (torch.sigmoid(logits) > 0.5).float().cpu().numpy()
    labels = (labels > 0.5).float().cpu().numpy()

    f1_list = []
    acc_list = [] # 정확도를 담을 바구니 추가

    for i in range(len(CRITERIA)):
        # 1. F1 Score 계산
        f1 = f1_score(labels[:, i], preds[:, i], average='macro', zero_division=0)
        f1_list.append(f1)

        # 2. Accuracy(정확도) 계산: 맞힌 개수 / 전체 개수
        acc = (labels[:, i] == preds[:, i]).mean()
        acc_list.append(acc)

    # 평균 F1, 상세 F1 리스트, 상세 정확도 리스트 3개를 돌려줍니다.
    return np.mean(f1_list), f1_list, acc_list

In [78]:
small_train_df = train_df.sample(n=40000, random_state=SEED).reset_index(drop=True)
small_val_df = val_df.sample(n=8000, random_state=SEED).reset_index(drop=True)

# 샘플용 데이터셋과 샘플러 다시 만들기
small_train_dataset = MultiHeadSoftDataset(small_train_df, tokenizer, MAX_LENGTH)
small_val_dataset = MultiHeadSoftDataset(small_val_df, tokenizer, MAX_LENGTH)
small_train_sampler = CurriculumSampler(small_train_df, total_epochs=NUM_EPOCHS)

샘플링

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# --- [준비 단계] 모델 및 도구 초기화 ---
model = MultiHeadModel(MODEL_NAME).to(device)
criterion = CombinedLoss()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# 데이터셋 및 샘플러 준비
train_dataset = MultiHeadSoftDataset(train_df, tokenizer, MAX_LENGTH)
val_dataset = MultiHeadSoftDataset(val_df, tokenizer, MAX_LENGTH)
train_sampler = CurriculumSampler(train_df, total_epochs=NUM_EPOCHS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- [함수 정의] 학습 및 검증 엔진 ---
def train_and_eval(model, criterion, optimizer, train_dataset, val_dataset, train_sampler):
    # 그래프를 그리기 위한 기록 바구니
    history = {'loss': [], 'f1': []}

    for epoch in range(NUM_EPOCHS):
        # [Curriculum] 에폭 설정 및 데이터로더 동적 생성
        train_sampler.set_epoch(epoch)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)

        # [A] 학습 단계 (Training)
        model.train()
        epoch_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch} Training"):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        history['loss'].append(avg_loss)

        # [B] 검증 단계 (Validation)
        model.eval()
        all_logits = []
        all_labels = []

        with torch.no_grad():
            for batch in val_loader:
                logits = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
                all_logits.append(logits.cpu()) # 메모리 관리를 위해 cpu로 이동
                all_labels.append(batch['labels'].cpu())

        # 9개 지표의 상세 성적표 계산
        all_logits = torch.cat(all_logits, dim=0)
        all_labels = torch.cat(all_labels, dim=0)

        # 앞서 만든 채점기 호출
        mean_f1, f1_list, acc_list = calculate_metrics(all_logits, all_labels)
        history['f1'].append(mean_f1)

        # --- 상세 성적표 출력 ---
        print(f"\n" + "="*50)
        print(f"[Epoch {epoch}] 상세 성적표")
        print("-" * 50)
        for i, criterion_name in enumerate(CRITERIA):
            print(f"{criterion_name:25}: F1={f1_list[i]:.4f} | Acc={acc_list[i]:.2%}")
        print("-" * 50)
        print(f"전체 평균 F1: {mean_f1:.4f} | Loss: {avg_loss:.4f}")
        print("="*50 + "\n")

    return history # 학습 기록 반환

# 작은 데이터로 엔진 가동 테스트
print("--- [맛보기 학습] 4,0000개 샘플로 테스트 시작 ---")
# --- [실행]

history = train_and_eval(
    model,
    criterion,
    optimizer,
    small_train_dataset,
    small_val_dataset,
    small_train_sampler
)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- [맛보기 학습] 4,0000개 샘플로 테스트 시작 ---


Epoch 0 Training:   0%|          | 0/313 [00:00<?, ?it/s]


[Curriculum] Epoch 0: 난이도 3.0 이하 데이터 39941개 학습


In [53]:
def predict_detailed(question, response, model, tokenizer):
    model.eval()
    # 1. 입력 텍스트 결합 (학습 때와 동일한 포맷)
    input_text = f"{question} [SEP] {response}"

    # 2. 토큰화 및 GPU 이동
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    ).to(device)

    # 3. 모델 추론
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        # 확률로 변환 (0~1 사이 소수점)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
        # 0.5 기준으로 합격(1)/불합격(0) 판정
        preds = (probs > 0.5).astype(int)

    # 4. 결과 출력 (베이스 코드 스타일)
    print(f"\n" + "="*50)
    print(f"   [ AI 품질 평가 상세 추론 결과 ]")
    print("="*50)
    print(f"질문: {question}")
    print(f"응답: {response}")
    print("-" * 50)
    print(f"{'평가 항목':<25} | {'결과':<5} | {'확신도'}")
    print("-" * 50)

    for i, criterion in enumerate(CRITERIA):
        status = "✅ PASS" if preds[i] == 1 else "❌ FAIL"
        # 확신도가 높을수록 모델이 해당 결과에 자신있어 함
        confidence = probs[i] if preds[i] == 1 else (1 - probs[i])
        print(f"{criterion:<25} | {status:<5} | {probs[i]:.2%}")
    print("="*50 + "\n")

# --- 테스트 실행 예시 ---
q_test = "한국의 수도는 어디야?"
r_test = "한국의 수도는 부산입니다. 부산은 바다가 아주 아름다운 도시예요."

predict_detailed(q_test, r_test, model, tokenizer)


   [ AI 품질 평가 상세 추론 결과 ]
질문: 한국의 수도는 어디야?
응답: 한국의 수도는 부산입니다. 부산은 바다가 아주 아름다운 도시예요.
--------------------------------------------------
평가 항목                     | 결과    | 확신도
--------------------------------------------------
linguistic_acceptability  | ✅ PASS | 95.24%
consistency               | ✅ PASS | 92.41%
interestingness           | ✅ PASS | 96.05%
unbias                    | ✅ PASS | 98.16%
harmlessness              | ✅ PASS | 97.59%
no_hallucination          | ✅ PASS | 88.91%
understandability         | ✅ PASS | 94.63%
sensibleness              | ✅ PASS | 91.74%
specificity               | ✅ PASS | 97.30%

